İş Problemi (Business Problem)

 FLO satış ve pazarlama faaliyetleri için roadmap belirlemek istemektedir.
 
 Şirketin orta uzun vadeli plan yapabilmesi için var olan müşterilerin gelecekte şirkete sağlayacakları potansiyel değerin tahmin edilmesi gerekmektedir.



# Veri Seti Hikayesi

 Veri seti son alışverişlerini 2020 - 2021 yıllarında OmniChannel(hem online hem offline alışveriş yapan) olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

**master_id:** Eşsiz müşteri numarası

**order_channel :** Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile, Offline)

**last_order_channel :** En son alışverişin yapıldığı kanal

**first_order_date :** Müşterinin yaptığı ilk alışveriş tarihi

**last_order_date :** Müşterinin yaptığı son alışveriş tarihi

**last_order_date_online :** Muşterinin online platformda yaptığı son alışveriş tarihi

**last_order_date_offline :** Muşterinin offline platformda yaptığı son alışveriş tarihi

**order_num_total_ever_online :** Müşterinin online platformda yaptığı toplam alışveriş sayısı

**order_num_total_ever_offline :** Müşterinin offline'da yaptığı toplam alışveriş sayısı

**customer_value_total_ever_offline :** Müşterinin offline alışverişlerinde ödediği toplam ücret

**customer_value_total_ever_online :** Müşterinin online alışverişlerinde ödediği toplam ücret

**interested_in_categories_12 :** Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi

In [1]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
from sklearn.preprocessing import MinMaxScaler

#Outlier Tespit Etme Fonksiyonu
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.25)
    quartile3 = dataframe[variable].quantile(0.75)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

#Outlier Baskılama Etme Fonksiyonu
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = round(low_limit)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = round(up_limit)

In [2]:
df=pd.read_csv(r"C:\Users\ACER\Desktop\Miuul\CRM\FLOCLTVPrediction\flo_data_20k.csv")

In [3]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0000,2.0000,159.9700,1853.5800,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR]


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,3.1109,4.2256,1.0000,1.0000,2.0000,4.0000,200.0000
order_num_total_ever_offline,19945.0000,1.9139,2.0629,1.0000,1.0000,1.0000,2.0000,109.0000
customer_value_total_ever_offline,19945.0000,253.9226,301.5329,10.0000,99.9900,179.9800,319.9700,18119.1400
customer_value_total_ever_online,19945.0000,497.3217,832.6019,12.9900,149.9800,286.4600,578.4400,45220.1300


In [5]:
replace_with_thresholds(df,"order_num_total_ever_online")
replace_with_thresholds(df,"order_num_total_ever_offline")
replace_with_thresholds(df,"customer_value_total_ever_offline")
replace_with_thresholds(df,"customer_value_total_ever_online")


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0000,2.6886,2.1748,1.0000,1.0000,2.0000,4.0000,8.0000
order_num_total_ever_offline,19945.0000,1.7669,0.9980,1.0000,1.0000,1.0000,2.0000,4.0000
customer_value_total_ever_offline,19945.0000,232.4516,173.9674,10.0000,99.9900,179.9800,319.9700,650.0000
customer_value_total_ever_online,19945.0000,416.8516,352.8910,12.9900,149.9800,286.4600,578.4400,1221.0200


In [7]:
df["Order_num_total"]=df["order_num_total_ever_offline"]+df["order_num_total_ever_online"]

In [8]:
df["Customer_value_total"]=df["customer_value_total_ever_offline"]+df["customer_value_total_ever_online"]

In [9]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,Order_num_total,Customer_value_total
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0000,1.0000,139.9900,799.3800,[KADIN],5.0000,939.3700
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,8.0000,2.0000,159.9700,1221.0000,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",10.0000,1380.9700
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0000,2.0000,189.9700,395.3500,"[ERKEK, KADIN]",5.0000,585.3200
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0000,1.0000,39.9900,81.9800,"[AKTIFCOCUK, COCUK]",2.0000,121.9700
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0000,1.0000,49.9900,159.9900,[AKTIFSPOR],2.0000,209.9800


In [10]:
df.dtypes

master_id                             object
order_channel                         object
last_order_channel                    object
first_order_date                      object
last_order_date                       object
last_order_date_online                object
last_order_date_offline               object
order_num_total_ever_online          float64
order_num_total_ever_offline         float64
customer_value_total_ever_offline    float64
customer_value_total_ever_online     float64
interested_in_categories_12           object
Order_num_total                      float64
Customer_value_total                 float64
dtype: object

In [11]:
dates=df.columns[df.columns.str.contains("date")]
df[dates]=df[dates].astype("datetime64[ns]")

In [12]:
df.dtypes

master_id                                    object
order_channel                                object
last_order_channel                           object
first_order_date                     datetime64[ns]
last_order_date                      datetime64[ns]
last_order_date_online               datetime64[ns]
last_order_date_offline              datetime64[ns]
order_num_total_ever_online                 float64
order_num_total_ever_offline                float64
customer_value_total_ever_offline           float64
customer_value_total_ever_online            float64
interested_in_categories_12                  object
Order_num_total                             float64
Customer_value_total                        float64
dtype: object

In [46]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [47]:
today_date = dt.datetime(2021, 6, 2)

In [48]:
cltv_df = pd.DataFrame({"CustomerID": df["master_id"],
             "Recency": ((df["last_order_date"] - df["first_order_date"]).dt.days)/7,
             "T": ((today_date - df["first_order_date"]).dt.days)/7,
             "Frequency": df["Order_num_total"],
             "Monetary": df["Customer_value_total"] / df["Order_num_total"]})

In [49]:
cltv_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Recency,19945.0000,95.2635,74.5894,0.0000,50.4286,76.5714,109.4286,433.4286
T,19945.0000,114.6147,74.7710,0.8571,74.0000,93.1429,119.5714,437.2857
Frequency,19945.0000,4.4556,2.4211,2.0000,3.0000,4.0000,6.0000,12.0000
Monetary,19945.0000,149.1325,63.8995,22.4900,106.1533,139.9450,176.4783,765.4850


In [61]:
cltv_df

last_order_date            Order_num_total Customer_value_total
                                          <lambda_0> <lambda_1>        <lambda>             <lambda>
master_id                                                                                           
00016786-2f5a-11ea-bb80-000d3a38a36f               0         11          5.0000             776.0700
00034aaa-a838-11e9-a2fc-000d3a38a36f               0        299          3.0000             269.4700
000be838-85df-11ea-a90b-000d3a38a36f               0        214          4.0000             722.6900
000c1fe2-a8b7-11ea-8479-000d3a38a36f               0         28          7.0000             874.1600
000f5e3e-9dde-11ea-80cd-000d3a38a36f               0         21          7.0000            1520.9800
...                                              ...        ...             ...                  ...
fff1db94-afd9-11ea-b736-000d3a38a36f               0        184          5.0000             796.8700
fff4736a-60a4-11ea-8dd8-000d3a38a36f               0         68          2.0000             318.8700
fffacd34-ae14-11e9-a2fc-000d3a38a36f               0        258          6.0000             983.9200
fffacecc-ddc3-11e9-a848-000d3a38a36f               0        133          9.0000            1362.2000
fffe4b30-18e0-11ea-9213-000d3a38a36f               0        322          4.0000             524.6900

[19945 rows x 4 columns]

In [18]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df['Frequency'],
        cltv_df['Recency'],
        cltv_df['T'])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 107.91, b: 0.00, r: 4.69>

In [19]:
cltv_df["exp_sales_3_month"]=bgf.predict(4*3,
            cltv_df['Frequency'],
            cltv_df['Recency'],
            cltv_df['T']).sort_values(ascending=False)

In [20]:
cltv_df["exp_sales_6_month"]=bgf.predict(4*6,
            cltv_df['Frequency'],
            cltv_df['Recency'],
            cltv_df['T']).sort_values(ascending=False)

In [21]:
cltv_df

,CustomerID,Recency,T,Frequency,Monetary,exp_sales_3_month,exp_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.7143,5.0000,187.8740,0.8389,1.6777
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,225.0000,10.0000,138.0970,0.5295,1.0591
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,79.0000,5.0000,117.0640,0.6222,1.2443
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,21.0000,2.0000,60.9850,0.6228,1.2456
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.5714,2.0000,104.9900,0.3946,0.7891
...,...,...,...,...,...,...,...
19940,727e2b6e-ddd4-11e9-a848-000d3a38a36f,41.1429,88.5714,3.0000,133.9867,0.4697,0.9394
19941,25cd53d4-61bf-11ea-8dd8-000d3a38a36f,42.2857,65.4286,2.0000,195.2350,0.4632,0.9264
19942,8aea4c2a-d6fc-11e9-93bc-000d3a38a36f,88.7143,90.0000,3.0000,210.9800,0.4663,0.9326
19943,e50bb46c-ff30-11e9-a5e8-000d3a38a36f,98.4286,114.0000,5.0000,189.5960,0.5240,1.0481


In [22]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv_df['Frequency'], cltv_df['Monetary'])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.16, q: 0.47, v: 4.09>

In [23]:
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df['Frequency'],
                                                                             cltv_df['Monetary'])

In [24]:
cltv_df=cltv_df.set_index("CustomerID")

In [25]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df['Frequency'],
                                   cltv_df['Recency'],
                                   cltv_df['T'],
                                   cltv_df['Monetary'],
                                   time=6,  # 3 aylık
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)

cltv.head()

cltv = cltv.reset_index()

In [26]:


cltv_final = cltv_df.merge(cltv, on="CustomerID", how="left")
cltv_final.sort_values(by="clv", ascending=False).head(20)

,CustomerID,Recency,T,Frequency,Monetary,exp_sales_3_month,exp_sales_6_month,exp_average_value,clv
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.2857,9.5714,2.0000,650.2450,0.6834,1.3668,696.4502,998.7648
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.8571,8.0000,4.0000,467.7500,0.8997,1.7994,484.1208,914.0228
15123,635b5e0a-a686-11eb-a6d3-000d3a299ebf,2.0000,5.2857,3.0000,447.1100,0.8153,1.6306,468.2676,801.1237
10792,f1f89712-84e5-11eb-8a3c-000d3a38a36f,7.2857,11.4286,2.0000,522.4500,0.6728,1.3455,560.0032,790.5918
7173,8897f4a8-c793-11ea-b753-000d3a38a36f,40.1429,45.8571,3.0000,597.1433,0.6002,1.2004,624.9232,787.0498
7153,5a351e90-b984-11eb-a757-000d3a38a36f,0.0000,1.8571,2.0000,451.9850,0.7314,1.4629,484.7676,744.0510
8496,c1f0cd2c-8d51-11eb-8604-000d3a38a36f,0.0000,9.8571,3.0000,430.2700,0.7836,1.5673,450.6843,741.1128
14858,031b2954-6d28-11eb-99c4-000d3a38a36f,14.8571,15.7143,3.0000,450.6533,0.7465,1.4930,471.9673,739.3410
14833,b09765ae-29a1-11eb-b280-000d3a38a36f,1.5714,28.0000,2.0000,552.8100,0.5907,1.1815,592.4187,734.3811
4087,5ee23224-ad83-11ea-b736-000d3a38a36f,49.5714,50.5714,2.0000,631.1050,0.5066,1.0132,676.0144,718.6599


In [27]:
cltv_final["segment"] = pd.qcut(cltv_final["clv"], 4, labels=["D", "C", "B", "A"])
cltv_final.head()

,CustomerID,Recency,T,Frequency,Monetary,exp_sales_3_month,exp_sales_6_month,exp_average_value,clv,segment
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.0000,30.7143,5.0000,187.8740,0.8389,1.6777,193.6020,340.7980,A
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.8571,225.0000,10.0000,138.0970,0.5295,1.0591,140.2849,155.8845,B
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.2857,79.0000,5.0000,117.0640,0.6222,1.2443,120.9493,157.9066,B
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.5714,21.0000,2.0000,60.9850,0.6228,1.2456,67.2962,87.9523,D
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.1429,95.5714,2.0000,104.9900,0.3946,0.7891,114.2804,94.6226,D


In [28]:
cltv_final.groupby("segment").agg(
    {"count", "mean", "sum"})

Recency                          T                      Frequency                   Monetary                       exp_sales_3_month                  exp_sales_6_month                  exp_average_value                         clv                      
          count     mean         sum count     mean         sum     count   mean        sum    count     mean          sum             count   mean       sum             count   mean       sum             count     mean          sum count     mean          sum
segment                                                                                                                                                                                                                                                             
D          4987 138.7243 691817.8571  4987 161.9714 807751.4286      4987 3.7409 18656.0000     4987  92.3851  460724.3370              4987 0.4045 2017.3505              4987 0.8090 4034.7009              4987  97.7957  487706.9513  4987  79.4511  396222.7564
C          4986 100.8017 502597.4286  4986 120.7973 602295.1429      4986 4.3578 21728.0000     4986 126.9508  632976.5582              4986 0.4905 2445.3933              4986 0.9809 4890.7867              4986 133.2315  664292.4614  4986 131.7976  657142.7354
B          4986  81.9571 408637.8571  4986 100.5980 501581.8571      4986 4.7401 23634.0000     4986 158.5038  790299.8540              4986 0.5499 2741.7784              4986 1.0998 5483.5568              4986 165.6440  825901.0744  4986 183.8884  916867.6115
A          4986  59.5622 296977.2857  4986  75.0825 374361.5714      4986 4.9836 24848.0000     4986 218.7017 1090446.6073              4986 0.6416 3198.8211              4986 1.2831 6397.6421              4986 228.3416 1138511.0104  4986 292.3183 1457499.2166

In [29]:
cltv_final.to_excel("cltv_prediction.xlsx")